<a href="https://colab.research.google.com/github/heghiw/DeepLearningPetIdentification/blob/main/fine54.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
## 1st -  Download utility.py file from github repository
## 2nd - Imports all functions from utility.py

import requests

# Correct raw URL for the utility.py file
url = "https://raw.githubusercontent.com/avkaz/DeepLearningPetIdentification/main/utility.py"

# Fetch and save the file locally
response = requests.get(url)
with open("utility.py", "wb") as f:
    f.write(response.content)


import utility
print("utility.py downloaded successfully.")

import tensorflow as tf

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from io import BytesIO
import pandas as pd
from google.colab import files
!pip install faiss-cpu
from tensorflow.keras.applications import EfficientNetB0
import faiss
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import pandas

utility.py downloaded successfully.


In [82]:
model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

In [83]:
data = utility.get_data()

In [84]:
filtered_data = {k: v for k, v in data.items() if len(v['images']) > 3}

filter out pets for training, only using postings that have 3+ pictures for triplet loss

In [85]:
nelen=int( 0.8*len(filtered_data))

In [86]:
keys = list(filtered_data.keys())
train_keys, test_keys = train_test_split(keys, train_size=nelen, random_state=42)
train_data = {k: filtered_data[k] for k in train_keys}
test_data = {k: filtered_data[k] for k in test_keys}

In [87]:
len(test_data)

363

download preprocess

In [ ]:
def preprocess_images(data):
    images = []
    labels = []
    for pet_id, pet_info in data.items():
        for img_url in pet_info['images']:
            try:
                img = utility.download_and_preprocess_image(img_url)
                images.append(img)
                labels.append(pet_id)
            except Exception as e:
                print(f"Error processing image {img_url}: {e}")
    return tf.convert_to_tensor(images), labels

train_images, train_labels = preprocess_images(train_data)
test_images, test_labels = preprocess_images(test_data)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.losses import Loss
from sklearn.model_selection import train_test_split
import faiss
import requests
from PIL import Image
import io

# Assuming utility.py with download_and_preprocess_image is imported
# from utility import download_and_preprocess_image


def filter_pets_by_images(data, min_images=3):
    """Filters pets with at least 'min_images' images."""
    return {key: value for key, value in data.items() if len(value['images']) >= min_images}


def triplet_loss(margin=1.0):
    """Triplet Loss function."""
    def loss(y_true, y_pred):
        anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
        positive_distance = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
        negative_distance = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
        loss_value = tf.maximum(positive_distance - negative_distance + margin, 0.0)
        return tf.reduce_mean(loss_value)
    return loss


def create_model():
    """Create the EfficientNetB0 model for feature extraction."""
    base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
    inputs = tf.keras.Input(shape=(128, 128, 3))
    x = base_model(inputs)
    model = Model(inputs, x)
    return model


def generate_embeddings(model, image_urls):
    """Generate embeddings for the list of image URLs."""
    embeddings = []
    for url in image_urls:
        img = utility.download_and_preprocess_image(url, target_size=(128, 128))
        emb = model.predict(np.expand_dims(img, axis=0))
        embeddings.append(emb.flatten())
    return np.array(embeddings)

In [ ]:
# Filter pets with at least 3 images
filtered_data = filter_pets_by_images(data, min_images=3)
model = create_model()
# Prepare the data for training and testing
all_images = []
all_labels = []

for pet_key, pet_info in filtered_data.items():
    # Use the first 3 images for each pet as an example
    images = pet_info['images']
    embeddings = generate_embeddings(model, images)
    label = pet_key  # Using the pet's key as the label (you can modify this)

    # Append embeddings and corresponding label
    all_images.extend(embeddings)
    all_labels.extend([label] * len(embeddings))



In [ ]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(np.array(all_images), np.array(all_labels), test_size=0.2, random_state=42)




In [ ]:
# Initialize FAISS index for similarity search
index = faiss.IndexFlatL2(X_train.shape[1])  # L2 distance
index.add(X_train)  # Add training data embeddings to the FAISS index

# Build the model
model = create_model()
model.compile(optimizer='adam', loss=triplet_loss(margin=1.0))

# Train the model with Triplet Loss
model.fit(X_train, y_train, epochs=10, batch_size=32)

# For evaluation or similarity search
query = generate_embeddings(model, [X_test[0]])  # Example query: the first image from the test set
D, I = index.search(query, k=5)  # Search for the 5 nearest neighbors
print("Nearest neighbors (indices):", I)
print("Distances:", D)